In [1]:
 # Installations-1
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=5ee6bb3cc7573ec599755a712be0e9864cb14a4ba2dc427a30e943ba1be32232
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [2]:
#!pip download lifelines

In [3]:
#!pip install scikit-learn

from sklearn import metrics

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lifelines.utils import concordance_index
from lifelines import KaplanMeierFitter

In [5]:

import plotly.io as pio
pio.renderers.default = 'iframe'
pd.options.display.max_columns = None 

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [7]:
# Import dataset (df0)
df0=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')

In [8]:
df = df0.drop('ID', axis=1)

#### KaplanMeierFitter()
_The KaplanMeierFitter_ is a tool from the lifelines library in Python used for survival analysis. It allows you to create Kaplan-Meier estimates, which are used to estimate the survival function from lifetime data.  
It's a class provided by the lifelines library, which makes it easier to fit and plot survival curves.

In [9]:
# Combining 2 target variables (efs and efs_time) into One Target- 

def kaplan(data=df, time_col = 'efs_time', event_col='efs'):
    
    kmf = KaplanMeierFitter()
    kmf.fit(data[time_col], event_observed=data[event_col])
    return kmf.survival_function_at_times(df[time_col]).values.flatten()


df['target'] = kaplan(data=df)

In [10]:
df.shape

(28800, 60)

In [11]:
df = df.drop(columns=['efs', 'efs_time'], errors='ignore')

In [12]:
df.duplicated()[df.duplicated()==True]
    #> No duplicates found

Series([], dtype: bool)

## Handling Missing Variables

In [13]:
for col in df.select_dtypes(include='object').columns:
       df[col] = df[col].str.strip().str.lower().replace(
        {'n/a': None, 'na': None, 'nan': None, '-': None})

In [14]:
# cat_vars
cat_vars = [var for var in df.columns if df[var].dtype == "O"]

# num_vars
num_vars= [var for var in df.columns if df[var].dtype != "O" and var != 'target']

In [15]:
print('cat_vars: ',cat_vars,'\n')
print('num_vars: ',num_vars )

# Note that target is num_var

cat_vars:  ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status', 'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe', 'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match', 'race_group', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'cardiac', 'pulm_moderate'] 

num_vars:  ['hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6', 'hla_high_res_6', 'hla_high_res_10', 'hla_match_dqb1_high', 'hla_nmdp_6', 'hla_match_c_low', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'year_hct', 'hla_match_a_high', 'donor_age', 'hla_match_b_low', 'age_at_hct', 'hla_match_a_low', 'hla_match_b_high', 'comorbidity_score', 'karnofsky_score', 'hla_low_res_8', 'hla_match_drb1_high', 'hla_low_res_10']


In [16]:
# Filling missing values for numerical columns with their median except target column
df[num_vars] = df[num_vars].fillna(df[num_vars].median())

In [17]:
# Filling missing values for categorical columns with their mode
df[cat_vars] = df[cat_vars].apply(
    lambda col: col.fillna(col.mode()[0] if not col.mode().empty else 'unknown'))

In [18]:
# Factorise Categorical vars - 
for i in df[cat_vars]:
    df[i], _ = pd.factorize(df[i])

In [19]:
pd.set_option('display.max_columns', None)

df.head()

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,prim_disease_hct,hla_high_res_6,cmv_status,hla_high_res_10,hla_match_dqb1_high,tce_imm_match,hla_nmdp_6,hla_match_c_low,rituximab,hla_match_drb1_low,hla_match_dqb1_low,prod_type,cyto_score_detail,conditioning_intensity,ethnicity,year_hct,obesity,mrd_hct,in_vivo_tcd,tce_match,hla_match_a_high,hepatic_severe,donor_age,prior_tumor,hla_match_b_low,peptic_ulcer,age_at_hct,hla_match_a_low,gvhd_proph,rheum_issue,sex_match,hla_match_b_high,race_group,comorbidity_score,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,target
0,0,0,0,0,2.0,8.0,0,0,6.0,0,0,0,0,0,6.0,0,10.0,2.0,0,6.0,2.0,0,2.0,2.0,0,0,0,0,2016,0,0,0,0,2.0,0,40.063,0,2.0,0,9.942,2.0,0,0,0,2.0,0,0.0,90.0,0,0,0,0,8.0,0,2.0,0,10.0,0.458687
1,1,0,1,0,2.0,8.0,1,0,6.0,1,0,0,0,1,6.0,0,10.0,2.0,0,6.0,2.0,0,2.0,2.0,1,0,0,0,2008,0,1,1,0,2.0,0,72.290,0,2.0,0,43.705,2.0,1,0,1,2.0,1,3.0,90.0,0,0,1,0,8.0,0,2.0,1,10.0,0.847759
2,0,0,0,0,2.0,8.0,0,0,6.0,0,0,0,0,2,6.0,0,10.0,2.0,0,6.0,2.0,0,2.0,2.0,0,0,0,0,2019,0,0,0,0,2.0,0,40.063,0,2.0,0,33.997,2.0,2,0,2,2.0,0,0.0,90.0,0,0,1,0,8.0,0,2.0,0,10.0,0.462424
3,2,0,1,0,2.0,8.0,0,0,6.0,0,0,0,0,3,6.0,0,10.0,2.0,0,6.0,2.0,0,2.0,2.0,0,0,0,0,2009,0,1,1,0,2.0,0,29.230,0,2.0,0,43.245,2.0,3,0,3,2.0,2,0.0,90.0,1,0,0,0,8.0,0,2.0,0,10.0,0.456661
4,2,0,0,0,2.0,8.0,0,0,6.0,1,0,0,0,4,6.0,0,10.0,2.0,0,5.0,2.0,0,2.0,2.0,1,0,0,1,2018,0,0,0,0,2.0,0,56.810,0,2.0,0,29.740,2.0,4,0,0,2.0,3,1.0,90.0,0,0,1,1,8.0,0,2.0,0,10.0,0.464674


## Correlation

In [20]:
# Get list of feature columns (excluding target)

X = [col for col in df.columns if col != 'target']

for feature in X:
    correlation = df[feature].corr(df["target"])
    print('Correlation for ', feature, ' = ',correlation)

Correlation for  dri_score  =  -0.015663222896883162
Correlation for  psych_disturb  =  0.0445884478144293
Correlation for  cyto_score  =  0.005200974001809842
Correlation for  diabetes  =  0.06472635931119704
Correlation for  hla_match_c_high  =  -0.012607743434186506
Correlation for  hla_high_res_8  =  -0.021272383826416604
Correlation for  tbi_status  =  0.03962863015684236
Correlation for  arrhythmia  =  0.05401029510435935
Correlation for  hla_low_res_6  =  -0.01637095516160984
Correlation for  graft_type  =  0.14554016971025033
Correlation for  vent_hist  =  0.008114302625674821
Correlation for  renal_issue  =  -0.0014354737125927305
Correlation for  pulm_severe  =  0.08856949096233102
Correlation for  prim_disease_hct  =  0.022227791379227767
Correlation for  hla_high_res_6  =  -0.022485941112610923
Correlation for  cmv_status  =  -0.04700236873073342
Correlation for  hla_high_res_10  =  -0.014870926680981769
Correlation for  hla_match_dqb1_high  =  -0.011014935896483587
Correla

## K Fold Validation

In [21]:
df1=df0.copy()

In [22]:
df1['target']=df['target']

In [23]:
# Define features and target
X1 = df1.drop(columns=['efs', 'efs_time', 'ID', 'target', 'rituximab'], errors='ignore')
target = df1['target']

In [24]:
# Identify categorical and numerical columns
cat_vars1 = X1.select_dtypes(include=['object']).columns
num_vars1 = X1.select_dtypes(include=['int64', 'float64']).columns

In [25]:
# Preprocessing for numerical data: Imputation and Scaling
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [26]:
from sklearn.preprocessing import OneHotEncoder

In [27]:
# Preprocessing for categorical data: Imputation and One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [28]:
from sklearn.compose import ColumnTransformer

In [29]:
# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_vars1),
        ('cat', categorical_transformer, cat_vars1)
    ]
)

## Create Model Pipeline

In [30]:
from sklearn.ensemble import GradientBoostingRegressor

In [31]:
# Create the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingRegressor(random_state=42))
])

In [32]:
from lifelines.utils import concordance_index

In [33]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
c_indices = []

for train_index, val_index in kf.split(X1):
    X_train, X_val = X1.iloc[train_index], X1.iloc[val_index]
    y_train, y_val = target.iloc[train_index], target.iloc[val_index]

# Fit the model
model.fit(X_train, y_train)

# Predict probabilities
val_probabilities = model.predict(X_val)

# Calculate C-index
c_index = concordance_index(y_val, val_probabilities)
c_indices.append(c_index)

print(f"Mean Stratified C-Index: {np.mean(c_indices):.4f}")
    

Mean Stratified C-Index: 0.6321


In [34]:
# Train the model on the full dataset
model.fit(X1, target)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6', 'hla_high_res_6',
       'hla_high_res_10', 'hla_match_dqb1_high', 'hla_nmdp_6',
       'hla_match_c_low', 'hla_match_drb1_low', 'hla_match_dqb1_low',
       'year_hct', 'hla_...
       'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'obesity',
       'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe', 'prior_tumor',
       'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match', 'race_group',
       'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose',
       'cardiac', 'pulm_moderate'],
      dtype='object'))])),
                ('classifier', GradientBoostingRegressor(random_state=42))])

In [35]:
# Load the test data
test_data = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

# Predict probabilities for the test dataset
prediction = model.predict(test_data.drop(columns=['ID', 'rituximab'], errors='ignore'))

# Add predictions to the test dataset
test_data['prediction'] = prediction

# Save predictions to a new CSV file
output_file_path = 'submission.csv'
test_data[['ID', 'prediction']].to_csv(output_file_path, index=False)

print(f"Predictions saved to {output_file_path}")

Predictions saved to submission.csv
